In [1]:
! killall ukb_wsd
%set_env ACE_GRAMMAR erg.dat
from mrs_logic import *
from ulkb import *

No matching processes belonging to you were found
env: ACE_GRAMMAR=erg.dat


## The syllogism

In [2]:
A = next(solve('Every man is mortal.'))
B = next(solve('Socrates is a man.'))
C = next(solve('Socrates is mortal.'))

In [3]:
print(A.mrs, B.mrs, C.mrs, sep=f'\n{"="*80}\n')

[ TOP: h0
  INDEX: e2
  RELS: < [ _every_q LBL: h4 ARG0: x3 RSTR: h5 BODY: h6 ]
          [ _man_n_1 LBL: h7 ARG0: x3 ]
          [ _mortal_a_1 LBL: h1 ARG0: e2 ARG1: x3 ] >
  HCONS: < h0 qeq h1 h5 qeq h7 > ]
[ TOP: h0
  INDEX: e2
  RELS: < [ proper_q LBL: h4 ARG0: x3 RSTR: h5 BODY: h6 ]
          [ named LBL: h7 ARG0: x3 CARG: "Socrates" ]
          [ _be_v_id LBL: h1 ARG0: e2 ARG1: x3 ARG2: x9 ]
          [ _a_q LBL: h10 ARG0: x9 RSTR: h11 BODY: h12 ]
          [ _man_n_1 LBL: h13 ARG0: x9 ] >
  HCONS: < h0 qeq h1 h5 qeq h7 h11 qeq h13 > ]
[ TOP: h0
  INDEX: e2
  RELS: < [ proper_q LBL: h4 ARG0: x3 RSTR: h5 BODY: h6 ]
          [ named LBL: h7 ARG0: x3 CARG: "Socrates" ]
          [ _mortal_a_1 LBL: h1 ARG0: e2 ARG1: x3 ] >
  HCONS: < h0 qeq h1 h5 qeq h7 > ]


### Solution 1: Ignore evars and treat `_be_v_id` as equality

In [4]:
aty = TypeVariable('a')
settings.serializer.ulkb.show_types = False
settings.serializer.ulkb.show_annotations = False 

In [5]:
PA = A.to_ulkb(universe_type=aty, drop_evars=True)
print(PA.serialize(show_types=True))

(∀ (x3 : a), (_man_n_1 : a → 𝔹) (x3 : a) → (_mortal_a_1 : a → 𝔹) (x3 : a)) : 𝔹


In [6]:
PB = B.to_ulkb(universe_type=aty, drop_evars=True, 
               translate_quantifiers = True,
               translate_predicates = True, mk_p_equal = ['_be_v_id'])
print(PB)

(∃ x9, _man_n_1 x9 ∧ (∃ x3, Socrates = x3 ∧ x3 = x9)) : 𝔹


In [19]:
PC = C.to_ulkb(universe_type=aty, drop_evars=True)
print(PC)

(∃ x3, Socrates = x3 ∧ _mortal_a_1 x3) : 𝔹


In [8]:
RuleZ3(Implies(PA, PB, PC))

⊢ (∀ x3, _man_n_1 x3 → _mortal_a_1 x3) → (∃ x9, _man_n_1 x9 ∧ (∃ x3, Socrates = x3 ∧ x3 = x9)) → (∃ x3, Socrates = x3 ∧ _mortal_a_1 x3)

### Solution 2: Handle the evars and `_be_v_id` in auxiliary axioms

In [9]:
PA = A.to_ulkb(universe_type=aty)
print(PA)

(∀ x3, _man_n_1 x3 → (∃ e2, _mortal_a_1 e2 x3)) : 𝔹


In [10]:
PB = B.to_ulkb(universe_type=aty)
print(PB)

(∃ x9, _man_n_1 x9 ∧ (∃ x3, Socrates = x3 ∧ (∃ e2, _be_v_id e2 x3 x9))) : 𝔹


In [11]:
PC = C.to_ulkb(universe_type=aty)
print(PC)

(∃ x3, Socrates = x3 ∧ (∃ e2, _mortal_a_1 e2 x3)) : 𝔹


In [12]:
new_axiom('ax_PA', PA)
new_axiom('ax_PB', PB)
show_axioms()

19	axiom ax_PA ≔ ∀ x3, _man_n_1 x3 → (∃ e2, _mortal_a_1 e2 x3)
20	axiom ax_PB ≔ ∃ x9, _man_n_1 x9 ∧ (∃ x3, Socrates = x3 ∧ (∃ e2, _be_v_id e2 x3 x9))


In [13]:
_be_v_id = new_constant('_be_v_id', FunctionType(aty, aty, aty, bool))

In [14]:
ei, ej, x, y = Variables('ei', 'ej', 'x', 'y', aty)
new_axiom('ax__be_v_id', Forall(ei, x, y, Implies(_be_v_id(ei, x, y), Equal(x, y))))
show_axioms()

19	axiom ax_PA ≔ ∀ x3, _man_n_1 x3 → (∃ e2, _mortal_a_1 e2 x3)
20	axiom ax_PB ≔ ∃ x9, _man_n_1 x9 ∧ (∃ x3, Socrates = x3 ∧ (∃ e2, _be_v_id e2 x3 x9))
22	axiom ax__be_v_id ≔ ∀ ei x y, _be_v_id ei x y → x = y


In [15]:
RuleZ3(PC)

⊢ ∃ x3, Socrates = x3 ∧ (∃ e2, _mortal_a_1 e2 x3)